In [104]:
from tqdm import trange
import numpy as np
import scipy.stats
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

# Grid approximation

In [74]:
n_points = 20

p_grid = np.linspace(0, 1, n_points)

prior = np.ones(n_points)

likelihood = np.array([scipy.stats.binom_test(6, 9, p) for p in p_grid])

posterior = likelihood * prior

posterior = posterior / np.sum(posterior)

p = figure(title='grid approximation - uniform prior', plot_width=400, plot_height=300)
p.line(p_grid, posterior)
show(p)

In [75]:
n_points = 20

p_grid = np.linspace(0, 1, n_points)

prior = p_grid >= 0.5

likelihood = np.array([scipy.stats.binom_test(6, 9, p) for p in p_grid])

posterior = likelihood * prior

posterior = posterior / np.sum(posterior)

p = figure(title='grid approximation - prior p_grid >= 0.5', plot_width=400, plot_height=300)
p.line(p_grid, posterior)
show(p)

In [78]:
n_points = 20

p_grid = np.linspace(0, 1, n_points)

prior = np.exp(-5 * np.abs(p_grid - 0.5))

likelihood = np.array([scipy.stats.binom_test(6, 9, p) for p in p_grid])

posterior = likelihood * prior

posterior = posterior / np.sum(posterior)

p = figure(title='grid approximation - prior exp', plot_width=400, plot_height=300)
p.line(p_grid, posterior)
show(p)

# Quadratic Approximation

# MCMC

In [ ]:
scipy.stats.norm.logpdf

In [125]:
def mcmc(data, n_samples):
    mu_prior = 0.0
    sigma_prior = 10.0
    mu_current = 10.0
    for i in trange(n_samples):
        mu_candidate = scipy.stats.norm.rvs(mu_current)
        
        log_likelihood_current = scipy.stats.norm(mu_current, 1).logpdf(data).sum()
        log_likelihood_candidate = scipy.stats.norm(mu_candidate, 1).logpdf(data).sum()
        
        log_prior_current = scipy.stats.norm(mu_prior, sigma_prior).logpdf(mu_current)
        log_prior_candidate = scipy.stats.norm(mu_prior, sigma_prior).logpdf(mu_candidate)
        
        log_posterior_current = log_likelihood_current + log_prior_current
        log_posterior_candidate = log_likelihood_candidate + log_prior_candidate
             
        if np.exp(log_posterior_current) > 0:
            ratio = np.exp(log_posterior_candidate) / np.exp(log_posterior_current)
        else:
            ratio = 1.0
        
        #print('ratio:', ratio)
        if np.random.rand() < ratio:
            mu_current = mu_candidate
            
        #print('----------------------')
    return mu_current

In [128]:
data = scipy.stats.norm.rvs(size=100)
mcmc(data, 10000)

100%|██████████| 10000/10000 [00:38<00:00, 257.18it/s]


-0.10522651241919878